# Task 2: Following Behaviours

### Step 1 load the pre-trained object detection model

Before running the program, we need to download the pre-trained detection model and put it in the current folder. The model can be found in the following link:https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view

In [1]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
    
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')


### Step 2 Initialize the camera instance for the Intel realsense sensor D435i

In [2]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of these two values will be captured by traitlets
    color_value = traitlets.Any()
    depth_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        self.stop_flag = False
        
        #start capture the first depth and color image
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        depth_frame = frames.get_depth_frame()           
        self.depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   
        
        self.warning_flag = 0
        self.min_distance = 0

    def _capture_frames(self):
        while(self.stop_flag==False):
            frames = self.pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            image = np.asanyarray(color_frame.get_data())
            self.color_value = image

            depth_frame = frames.get_depth_frame()              
            self.depth_image = np.asanyarray(depth_frame.get_data())
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap
            
            depths = self.depth_image[self.depth_image!=0]
            if depths.size == 0:
                self.min_distance = 0
            else:
                self.min_distance = depths.min()
            self.warning_flag = (self.min_distance < 200)
    
    def start(self):
        if not hasattr(self, 'thread') or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        self.stop_flag = True
        if hasattr(self, 'thread'):
            self.thread.join()        
        if self.pipeline_started:
            self.pipeline.stop()

def bgr8_to_jpeg(value,quality=75):
    return bytes(cv2.imencode('.jpg',value)[1])



In [3]:
#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

### Step 3 Perform object detection and display the results on widgets
Human is labeled is 1 in the pretrained model. A full list of the detection class indices can be found in the following link https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt

The following program will ask the robot to move forward if a human is detected in the captured color image.

In [4]:
from mochris import RangeSensors
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
from ui import obstacle_slider

# Range sensors
range_sensors = RangeSensors(front=True)
def update(change):
    front_range = change['new']    
    obstacle_slider.value = front_range
range_sensors.observe(update, names='front_value')

In [12]:
from ipywidgets import widgets, Layout
from IPython.display import display
from random import random
from ui import image_widget, label_widget, distance_widget, obj_widget, target_widget, similarity_widget, obstacle_slider
from ui import start_button, track_button, stop_button

width = 640
height = 480

# Creates the buttons used to control the robot
buttons = widgets.VBox([
    start_button, 
    track_button,
    stop_button
], layout=Layout(flex='1 1 auto', width='auto'))

# Creates the sliders used to control the distance, obstacle and similarity.
sliders = widgets.VBox([
    distance_widget, 
    obstacle_slider, 
    similarity_widget
], layout=Layout(flex='6 1 auto', width='auto'))

# Handles stop button press.
# When the button is clicked, the camera and robot are stopped.
def stop_button_clicked(b):
    camera.unobserve_all()
    time.sleep(1.0)
    robot.stop()

# Handles the start button press.
# When the button is clicked, the camera is turned on.
def start_button_clicked(b):
    camera.observe(processing, names=['color_value'])
    
# Handles the track button clicked. 
# When the button is clicked the target becomes the current object.
def track_button_clicked(b):
    global target
    global obj
    target = obj

# Hooks the buttons to their respective action handlers.    
start_button.on_click(start_button_clicked)  
stop_button.on_click(stop_button_clicked)   
track_button.on_click(track_button_clicked)

# Creates the windows that the camera, object image and target image will be displayed on.
display(widgets.VBox([
    widgets.HBox([image_widget, obj_widget, target_widget]),
    widgets.HBox([label_widget,]),
    widgets.HBox([buttons, sliders]),
]))


import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()

# Handles stopping the robot to allow the camera to be stabalised. 
# When this set, the robot should stop moving. 
# 0 = The robot is not stopped
# 1, 2, 3, etc = The robot is stopped
stop_flag = 0

# Sets the direction the robot will move.
# 0 = The robot will turn right otherwise the robot will turn left.
left_flag = 0

# The target object to follow when tracking.
# By default we start with an empty object.
target = np.ones((300, 300, 3)).astype(np.uint8)

# The current object that is in front of the camera during tracking.
# By default we start with an empty object.
obj = np.ones((300, 300, 3)).astype(np.uint8)

# Handles processing the robot. 
# This function controls the actions that the robot performs so for task 2,
# this function contains the logic for tracking an object. 
def processing(change):
    image = change['new']
    depth = camera.depth_image
    
    # The number of objects on the left hand side of the screen.  
    left_count = 0 
    
    # The number of objects in the middle of the screen.     
    center_count = 0
    
    # The number of objects in the center of the screen.     
    right_count = 0
    
    global stop_flag
    global left_flag
    global target
    global obj
     
    imgsized = cv2.resize(image,(300,300))
    
    # Compute all detected objects
    detections = model(imgsized)
    
    # Filter out all the objects that match the object we are tracking.      
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # Get the distances for all matching detections in the image    
    distances = [0 for d in matching_detections]
    
    # The largest simularity score 
    similarity_largest = 0
    
    # The distance between the robot and object with the largest simularity
    distance_largest = 0
    
    # Center of the bounding box of object with largest simularity    
    center_largest = 0.5
    
    for i, det in enumerate(matching_detections):
        bbox = det['bbox']
        
        x1 = int(width*bbox[0])
        x2 = int(width*bbox[2])
        y1 = int(width*bbox[1])
        y2 = int(width*bbox[3])
        depths = depth[y1:y2, x1:x2]
        depths = depths[depths != 0]
        
        obj_temp = image[y1:y2, x1:x2, :]
        
        if obj_temp.size == 0:
            continue

        obj_temp = cv2.resize(obj_temp,(300,300))
        obj_hsv = cv2.cvtColor(obj_temp, cv2.COLOR_BGR2HSV)
        target_hsv = cv2.cvtColor(target, cv2.COLOR_BGR2HSV)
        
        # Calculates the number of differences between the tracked object and current object.         
        similarity = 1 - np.mean(np.absolute(obj_hsv/255.0 - target_hsv/255.0))
        
        # If the current object has a larger simularity than the previous object,
        # update the tracked image with the better image for tracking.          
        if similarity > similarity_largest:
            similarity_largest = similarity
            obj = obj_temp
            if depths.size == 0:
                distance_largest = 0
            else:
                distance_largest = int(np.nanmin(depths))
            center_largest = (bbox[0] + bbox[2]) / 2.0
        
        # Draws a bounding box around detected object        
        if similarity > 0.6:
            cv2.rectangle(image, (x1, y1, x2, y2), (255, 0, 0), 2)
            continue
        else:
            cv2.rectangle(image, (x1, y1, x2, y2), (0, 0, 255), 2)
            
    if similarity_largest > 0.99:
        target = obj
    
    similarity_widget.value = similarity_largest
    distance_widget.value = distance_largest

    # Wander Behaviour    
    if similarity_largest < 0.7:
        if random() < 0.5:
            if stop_flag:
                stop_flag = stop_flag - 1
                robot.stop()
            elif left_flag:
                robot.left(0.5)
            else:
                robot.right(0.5)
        else:
            robot.stop()
    else:
        # Person Following Behaviour        
        if center_largest < 1/3.0:
            stop_flag = 3
            left_flag = 1
            robot.left(0.5)
        elif center_largest > 2/3.0:
            stop_flag = 3
            left_flag = 0
            robot.right(0.5)
        # Collision Avoidance        
        elif distance_largest < 300:
            robot.backward(0.8)
        elif distance_largest < 600:
            robot.stop()
        else:
            robot.forward(0.3 + (distance_largest-600)/1000*0.5)
    
    image_widget.value = bgr8_to_jpeg(image)
    target_widget.value = bgr8_to_jpeg(target)
    obj_widget.value = bgr8_to_jpeg(obj)
    

In [ ]:
camera.stop()

The following code can be used to stop the capturing of the image and the moving of the robot